# Imports

In [1]:
import pandas as pd
import numpy as np
import json
import requests

**installing spotipy**

In [2]:
!pip install spotipy

In [3]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

# Setting the credencials

In [8]:
client_id = %env CLIENT_ID
client_secret = %env CLIENT_SECRET

sp = spotipy.Spotify(client_credentials_manager=SpotifyClientCredentials(client_id=client_id, client_secret=client_secret))


# Prepare data

In [263]:
# Playlist id
pl_id = 'spotify:playlist:56r5qRUv3jSxADdmBkhcz7'

# Getting the tracks in the playlist
response = sp.playlist(pl_id)

# How many tracks in the playlist
len(response['tracks']['items'])

100

In [264]:
 response['tracks']['items'][1].keys()

dict_keys(['added_at', 'added_by', 'is_local', 'primary_color', 'track', 'video_thumbnail'])

## Getting features ready to turn into dataframe

### Set basics features to build a basic dataset

In [265]:
name = [response['tracks']['items'][i]['track']['name'] for i in range(len(response['tracks']['items']))]

In [266]:
artist = [response['tracks']['items'][i]['track']['artists'][0]['name'] for i in range(len(response['tracks']['items']))]

In [267]:
artist_id = [response['tracks']['items'][i]['track']['album']['artists'][0]['id'] for i in range(len(response['tracks']['items']))]

In [268]:
popularity = [response['tracks']['items'][i]['track']['popularity'] for i in range(len(response['tracks']['items']))]

In [269]:
Id = [response['tracks']['items'][i]['track']['id'] for i in range(len(response['tracks']['items']))]

In [270]:
release_date = [response['tracks']['items'][i]['track']['album']['release_date'] for i in range(len(response['tracks']['items']))]

In [271]:
preview_url = [response['tracks']['items'][i]['track']['preview_url'] for i in range(len(response['tracks']['items']))]

In [272]:
cover_album = [response['tracks']['items'][i]['track']['album']['images'][1]['url'] for i in range(len(response['tracks']['items']))]

In [273]:
explicit = [response['tracks']['items'][i]['track']['explicit'] for i in range(len(response['tracks']['items']))]

### Features Dataframe ready

In [274]:
data = {}

# Setting the features
data['id'] = Id
data['name'] = name
data['popularity'] = popularity
data['artists'] = artist
data['id_artists'] = artist_id
data['explicit'] = explicit

data['release_date'] = release_date
data['preview_url'] = preview_url
data['cover_album'] = cover_album

# turning into dataframe
df = pd.DataFrame(data)

#turninge the realise date into datetime
df['release_date'] = pd.to_datetime(df['release_date'])

# Turning explicit column in 0 or 1
df['explicit'] = df['explicit'].map(lambda x: 1 if x == True else 0)

### Audio features  Datarame ready

In [275]:
# List of features set
tracks = []

# For each track in the 1st dataframe 
for track_id in df.loc[:,'id']:
    track = f'spotify:track:{track_id}' # Get the track features
    audio_features = sp.audio_features(track) # Save in audio features variable
    tracks.append(audio_features[0]) # Append to the tracks list

In [276]:
# New dataframe with all the tracks in the first dataframe
df_features =  pd.DataFrame(tracks)
df_features.head()

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,0.520,0.731,6,-5.338,0,0.0557,0.3420,0.001010,0.3110,0.662,173.930,audio_features,4LRPiXqCikLlN15c3yImP7,spotify:track:4LRPiXqCikLlN15c3yImP7,https://api.spotify.com/v1/tracks/4LRPiXqCikLl...,https://api.spotify.com/v1/audio-analysis/4LRP...,167303,4
1,0.637,0.643,4,-6.571,1,0.0519,0.1300,0.000002,0.1420,0.533,97.008,audio_features,0V3wPSX9ygBnCm8psDIegu,spotify:track:0V3wPSX9ygBnCm8psDIegu,https://api.spotify.com/v1/tracks/0V3wPSX9ygBn...,https://api.spotify.com/v1/audio-analysis/0V3w...,200690,4
2,0.714,0.472,2,-7.375,1,0.0864,0.0130,0.000005,0.2660,0.238,131.121,audio_features,3nqQXoyQOWXiESFLlDF1hG,spotify:track:3nqQXoyQOWXiESFLlDF1hG,https://api.spotify.com/v1/tracks/3nqQXoyQOWXi...,https://api.spotify.com/v1/audio-analysis/3nqQ...,156943,4
3,0.905,0.563,8,-6.135,1,0.1020,0.0254,0.000010,0.1130,0.324,106.998,audio_features,1rDQ4oMwGJI7B4tovsBOxc,spotify:track:1rDQ4oMwGJI7B4tovsBOxc,https://api.spotify.com/v1/tracks/1rDQ4oMwGJI7...,https://api.spotify.com/v1/audio-analysis/1rDQ...,173948,4
4,0.625,0.533,10,-11.903,0,0.0596,0.6590,0.002660,0.0546,0.139,108.296,audio_features,29d0nY7TzCoi22XBqDQkiP,spotify:track:29d0nY7TzCoi22XBqDQkiP,https://api.spotify.com/v1/tracks/29d0nY7TzCoi...,https://api.spotify.com/v1/audio-analysis/29d0...,300840,4


### Merge and Cleaning

In [277]:
# Inner merging between the 2 dataframe on id
data = df.merge(df_features, on='id')

# Dropping columns 
data.drop(['type', 'uri', 'track_href', 'analysis_url', 'time_signature'], axis=1, inplace=True)

data.shape

(100, 21)

### Getting the genre and the follower for each track in the playlist

In [319]:
# Creating the dataframe base with a dict
artista = {'id': [], 'genre': [], 'followers': [], 'artist_popularity': []}

# For each artist id get the id, genre and followers and append to the dict
for _id in data['id_artists']:
    artist = sp.artist(_id)
    artista['id'].append(artist['id'])
    artista['genre'].append(artist['genres'])
    artista['followers'].append(artist['followers']['total'])
    artista['artist_popularity'].append(artist['popularity'])
    
artista

{'id': ['6KImCVD70vtIoJWnq6nGn3',
  '06HL4z0CvFAxyc27GXpf02',
  '2wY79sveU1sp5g7SokKOiI',
  '2LIk90788K0zvyj2JJVwkJ',
  '1aSxMhuvixZ8h9dK9jIDwL',
  '7jVv8c5Fj3E9VhNjxT4snq',
  '6vWDO969PvNqNYHIOW5v0m',
  '56oDRnqbIiwx4mymNEv7dS',
  '4q3ewBCX7sLwd24euuV69X',
  '2feDdbD5araYcm6JhFHHw7',
  '57vWImR43h4CaDao012Ofp',
  '3MZsBdqDrRTJihTHQrO6Dq',
  '4yvcSjfu4PC0CYQyLy4wSq',
  '2VSHKHBTiXWplO8lxcnUC9',
  '7jVv8c5Fj3E9VhNjxT4snq',
  '6KImCVD70vtIoJWnq6nGn3',
  '4nDoRrQiYLoBzwC5BhVJzF',
  '1uNFoZAHBGtllmzznpCI3s',
  '4q3ewBCX7sLwd24euuV69X',
  '716NhGYqD1jl2wI1Qkgq36',
  '7ltDVBr6mKbRvohxheJ9h1',
  '7FNnA9vBm6EKceENgCGRMb',
  '0hCNtLu0JehylgoiP8L4Gh',
  '41MozSoPIsD1dJM0CLPjZF',
  '246dkjvS1zLTtiykXe5h60',
  '6eUKZXaKkcviH0Ku9w2n3V',
  '6VuMaDnrHyPL1p4EHjYLi7',
  '1McMsnEElThX1knmY4oliG',
  '7jVv8c5Fj3E9VhNjxT4snq',
  '2W8yFh0Ga6Yf3jiayVxwkE',
  '67MNhiAICFY6Pwc2YxCO0K',
  '53XhwfbYqKCa1cC15pYq2q',
  '06HL4z0CvFAxyc27GXpf02',
  '4dpARuHxo51G3z768sgnrY',
  '1Xyo4u8uXC1ZmMpatF05PJ',
  '3PhoLpVuITZ

In [320]:
# Turning dict into dataframe
data_artist = pd.DataFrame(artista)
data_artist.head()

,id,genre,followers,artist_popularity
0,6KImCVD70vtIoJWnq6nGn3,[pop],26642393,91
1,06HL4z0CvFAxyc27GXpf02,[pop],70161770,100
2,2wY79sveU1sp5g7SokKOiI,"[pop, uk pop]",22109406,89
3,2LIk90788K0zvyj2JJVwkJ,"[deep underground hip hop, kentucky hip hop, p...",2825648,82
4,1aSxMhuvixZ8h9dK9jIDwL,"[art pop, art rock, baroque pop, new wave pop,...",1651384,73


##### Checking shape, duplicated 

In [322]:
data_artist.shape

(100, 4)

In [323]:
data.duplicated()

0     False
1     False
2     False
3     False
4     False
      ...  
95    False
96    False
97    False
98    False
99    False
Length: 100, dtype: bool

#### Getting the artist dataframe ready to merge with tha main one 

In [321]:
data_artist.info() # Dtype of each column

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 4 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   id                 100 non-null    object
 1   genre              100 non-null    object
 2   followers          100 non-null    int64 
 3   artist_popularity  100 non-null    int64 
dtypes: int64(2), object(2)
memory usage: 3.2+ KB


In [324]:
data_artist['followers'] = data_artist['followers'].astype('float') # Changing to int to float

In [325]:
data_artist.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 4 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 100 non-null    object 
 1   genre              100 non-null    object 
 2   followers          100 non-null    float64
 3   artist_popularity  100 non-null    int64  
dtypes: float64(1), int64(1), object(2)
memory usage: 3.2+ KB


In [326]:
# Selecting the genre columns , for each row remove brackets 
data_artist['genre'] = data_artist['genre'].apply(lambda x: ', '.join(map(str, x)))

In [327]:
data_artist.head()

,id,genre,followers,artist_popularity
0,6KImCVD70vtIoJWnq6nGn3,pop,26642393.0,91
1,06HL4z0CvFAxyc27GXpf02,pop,70161770.0,100
2,2wY79sveU1sp5g7SokKOiI,"pop, uk pop",22109406.0,89
3,2LIk90788K0zvyj2JJVwkJ,"deep underground hip hop, kentucky hip hop, po...",2825648.0,82
4,1aSxMhuvixZ8h9dK9jIDwL,"art pop, art rock, baroque pop, new wave pop, ...",1651384.0,73


In [328]:
data_artist.shape

(100, 4)

In [329]:
# Cahnge the id to id_artists
data_artist.rename(columns={'id': 'id_artists'}, inplace=True)

In [342]:
# Merging the playlist with the artists
data_nova = data_artist.merge(data, on='id_artists', )

In [343]:
# Rremoving duplicated
data_nova.drop_duplicates(inplace=True)

In [344]:
data_nova.head()

,id_artists,genre,followers,artist_popularity,id,name,popularity,artists,explicit,release_date,...,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms
0,6KImCVD70vtIoJWnq6nGn3,pop,26642393.0,91,4LRPiXqCikLlN15c3yImP7,As It Was,90,Harry Styles,0,2022-03-31,...,6,-5.338,0,0.0557,0.342,0.001010,0.311,0.6620,173.930,167303
1,6KImCVD70vtIoJWnq6nGn3,pop,26642393.0,91,1qEmFfgcLObUfQm0j1W2CK,Late Night Talking,87,Harry Styles,0,2022-05-20,...,10,-4.595,1,0.0468,0.298,0.000000,0.106,0.9010,114.996,177955
4,06HL4z0CvFAxyc27GXpf02,pop,70161770.0,100,0V3wPSX9ygBnCm8psDIegu,Anti-Hero,93,Taylor Swift,0,2022-10-21,...,4,-6.571,1,0.0519,0.130,0.000002,0.142,0.5330,97.008,200690
5,06HL4z0CvFAxyc27GXpf02,pop,70161770.0,100,5jQI2r1RdgtuT8S3iG8zFC,Lavender Haze,86,Taylor Swift,1,2022-10-21,...,10,-10.489,1,0.0800,0.258,0.000573,0.157,0.0976,96.985,202396
6,06HL4z0CvFAxyc27GXpf02,pop,70161770.0,100,1wtOxkiel43cVs0Yux5Q4h,Snow On The Beach (feat. Lana Del Rey),80,Taylor Swift,1,2022-10-21,...,9,-13.481,1,0.0375,0.690,0.000993,0.117,0.1930,109.957,256124


In [345]:
data_nova.columns

Index(['id_artists', 'genre', 'followers', 'artist_popularity', 'id', 'name',
       'popularity', 'artists', 'explicit', 'release_date', 'preview_url',
       'cover_album', 'danceability', 'energy', 'key', 'loudness', 'mode',
       'speechiness', 'acousticness', 'instrumentalness', 'liveness',
       'valence', 'tempo', 'duration_ms'],
      dtype='object')

# Merging the both dataset

In [304]:
kaggle = pd.read_csv('/home/mdbruchard/code/dmassonfr/Spotify_hit_predictor/raw_data/kaggle_df.csv')

In [307]:
#Cheking the shape
kaggle.shape

(586672, 23)

In [310]:
# Duplicated ?
kaggle.duplicated().sum()

0

In [313]:
kaggle.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 586672 entries, 0 to 586671
Data columns (total 23 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   Unnamed: 0         586672 non-null  int64  
 1   track_id           586672 non-null  object 
 2   track_name         586601 non-null  object 
 3   track_popularity   586672 non-null  int64  
 4   release_date       586672 non-null  object 
 5   duration_ms        586672 non-null  int64  
 6   artists            586672 non-null  object 
 7   artists_ids        586672 non-null  object 
 8   first_artist       586672 non-null  object 
 9   artist_popularity  574812 non-null  float64
 10  followers          574811 non-null  float64
 11  genres             574812 non-null  object 
 12  danceability       586672 non-null  float64
 13  energy             586672 non-null  float64
 14  key                586672 non-null  int64  
 15  loudness           586672 non-null  float64
 16  mo

In [347]:
# Which columns doesn't match or have different name
for col in data_nova.columns:
    if col not in kaggle.columns:
        print(col)

id_artists
genre
id
name
popularity
explicit
preview_url
cover_album


For those columns that doesn't match by name I'll change the name so then I cant merge them by a list of columns. Those columns that doesn't exist on the kaggle dataset I'll merge them together, if they aren't needed we can drop them later

In [350]:
# Renaming all columns necessary 
data_nova.rename(columns={
    'id_artists': 'artists_ids', 
    'genre': 'genres', 
    'id': 'track_id', 
    'name': 'track_name', 
    'popularity': 'track_popularity'}, inplace=True)

In [359]:
# Drop column of index duplicated
kaggle.drop(columns='Unnamed: 0', inplace=True)

In [365]:
# Turn Dtype release date into datetime
kaggle['release_date'] = pd.to_datetime(kaggle['release_date'])

In [366]:
kaggle.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 586672 entries, 0 to 586671
Data columns (total 22 columns):
 #   Column             Non-Null Count   Dtype         
---  ------             --------------   -----         
 0   track_id           586672 non-null  object        
 1   track_name         586601 non-null  object        
 2   track_popularity   586672 non-null  int64         
 3   release_date       586672 non-null  datetime64[ns]
 4   duration_ms        586672 non-null  int64         
 5   artists            586672 non-null  object        
 6   artists_ids        586672 non-null  object        
 7   first_artist       586672 non-null  object        
 8   artist_popularity  574812 non-null  float64       
 9   followers          574811 non-null  float64       
 10  genres             574812 non-null  object        
 11  danceability       586672 non-null  float64       
 12  energy             586672 non-null  float64       
 13  key                586672 non-null  int64   

In [368]:
# For columns in dataset from API
# Add the Dataset API columns in a list of columns 
# If the column is also in the kaggle dataframe
colunas = [col for col in data_nova.columns if col in kaggle.columns]
colunas

['artists_ids',
 'genres',
 'followers',
 'artist_popularity',
 'track_id',
 'track_name',
 'track_popularity',
 'artists',
 'release_date',
 'danceability',
 'energy',
 'key',
 'loudness',
 'mode',
 'speechiness',
 'acousticness',
 'instrumentalness',
 'liveness',
 'valence',
 'tempo',
 'duration_ms']

In [371]:
# Merge the both dataset combine all the columns in common
data_combined = kaggle.merge(data_nova, on=colunas, how='outer')

In [375]:
# Make sure the merge is correct 
assert(data_combined.shape[0] - data_nova.shape[0] == kaggle.shape[0])

In [377]:
data_combined.head()

,track_id,track_name,track_popularity,release_date,duration_ms,artists,artists_ids,first_artist,artist_popularity,followers,...,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,explicit,preview_url,cover_album
0,35iwgR4jXetI318WEWsa1Q,Carve,6,1922-02-22,126903,['Uli'],['45tIt06XoI0Iio4LBEVpls'],45tIt06XoI0Iio4LBEVpls,4.0,91.0,...,1,0.4510,0.674,0.7440,0.151,0.127,104.851,NaN,NaN,NaN
1,021ht4sdgPcrDgSk7JTbKY,Capítulo 2.16 - Banquero Anarquista,0,1922-06-01,98200,['Fernando Pessoa'],['14jtPCOoNZwquk5wd9DxrY'],14jtPCOoNZwquk5wd9DxrY,0.0,3.0,...,1,0.9570,0.797,0.0000,0.148,0.655,102.009,NaN,NaN,NaN
2,07A5yehtSnoedViJAZkNnc,Vivo para Quererte - Remasterizado,0,1922-03-21,181640,['Ignacio Corsini'],['5LiOoJbxVSAMkBS2fUm3X2'],5LiOoJbxVSAMkBS2fUm3X2,23.0,3528.0,...,1,0.0512,0.994,0.0218,0.212,0.457,130.418,NaN,NaN,NaN
3,08FmqUhxtyLTn6pAh6bk45,El Prisionero - Remasterizado,0,1922-03-21,176907,['Ignacio Corsini'],['5LiOoJbxVSAMkBS2fUm3X2'],5LiOoJbxVSAMkBS2fUm3X2,23.0,3528.0,...,1,0.0504,0.995,0.9180,0.104,0.397,169.980,NaN,NaN,NaN
4,08y9GfoqCWfOGsKdwojr5e,Lady of the Evening,0,1922-01-01,163080,['Dick Haymes'],['3BiJGZsyX9sJchTqcSA7Su'],3BiJGZsyX9sJchTqcSA7Su,35.0,11327.0,...,0,0.0390,0.989,0.1300,0.311,0.196,103.220,NaN,NaN,NaN


# Saving the new dataset as CSV 

In [376]:
data_combined.to_csv('/home/mdbruchard/code/dmassonfr/Spotify_hit_predictor/raw_data/tracks_w_api.csv')

# Kaggle dataset to compare with API dataset

In [133]:
# Get the data from kaggle to compare
df_tracks = pd.read_csv('/home/mdbruchard/code/dmassonfr/Spotify_hit_predictor/raw_data/tracks.csv')
df_artists = pd.read_csv('/home/mdbruchard/code/dmassonfr/Spotify_hit_predictor/raw_data/artists.csv')
df_tracks.head()

,id,name,popularity,duration_ms,explicit,artists,id_artists,release_date,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature
0,35iwgR4jXetI318WEWsa1Q,Carve,6,126903,0,['Uli'],['45tIt06XoI0Iio4LBEVpls'],1922-02-22,0.645,0.4450,0,-13.338,1,0.4510,0.674,0.7440,0.151,0.127,104.851,3
1,021ht4sdgPcrDgSk7JTbKY,Capítulo 2.16 - Banquero Anarquista,0,98200,0,['Fernando Pessoa'],['14jtPCOoNZwquk5wd9DxrY'],1922-06-01,0.695,0.2630,0,-22.136,1,0.9570,0.797,0.0000,0.148,0.655,102.009,1
2,07A5yehtSnoedViJAZkNnc,Vivo para Quererte - Remasterizado,0,181640,0,['Ignacio Corsini'],['5LiOoJbxVSAMkBS2fUm3X2'],1922-03-21,0.434,0.1770,1,-21.180,1,0.0512,0.994,0.0218,0.212,0.457,130.418,5
3,08FmqUhxtyLTn6pAh6bk45,El Prisionero - Remasterizado,0,176907,0,['Ignacio Corsini'],['5LiOoJbxVSAMkBS2fUm3X2'],1922-03-21,0.321,0.0946,7,-27.961,1,0.0504,0.995,0.9180,0.104,0.397,169.980,3
4,08y9GfoqCWfOGsKdwojr5e,Lady of the Evening,0,163080,0,['Dick Haymes'],['3BiJGZsyX9sJchTqcSA7Su'],1922,0.402,0.1580,3,-16.900,0,0.0390,0.989,0.1300,0.311,0.196,103.220,4


In [115]:
df_tracks['id_artists'] = df_tracks['id_artists'].map(lambda x: x.strip("'[]'"))
df_tracks.drop(columns=['time_signature'], inplace=True)

In [119]:
df_tracks['release_date'] = pd.to_datetime(df_tracks['release_date'])

In [378]:
df_artists['genres'].unique()

array(['[]', "['carnaval cadiz']", "['classical harp', 'harp']", ...,
       "['dark post-punk', 'french post-punk', 'polish post-punk']",
       "['pop punk', 'tulsa indie']",
       "['clean comedy', 'deep comedy', 'new comedy']"], dtype=object)